In [1]:
%pip install pandas numpy scikit-learn 

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 17.7 kB/s eta 0:00:00m eta 0:00:010:00:19
     ━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/18.2 MB 13.6 kB/s eta 0:15:56^C7
     ━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/18.2 MB 13.6 kB/s eta 0:15:56
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'pandas'

In [ ]:
pd.options.display.max_columns = None # Display all columns

In [ ]:
%ls

In [ ]:
dataset = pd.read_csv("data/Joshua_Tree_Airbnb_Raw.csv")

In [ ]:
dataset.head(3)

Identifying Columns of Interest
- Address
- Name
- Reviews/0-9/comments
It seems this dataset do not contain any column that has a description of of the corresponding hotel. Lets try another airbnb dataset. More specifically,[[https://www.kaggle.com/datasets/tylerx/melbourne-airbnb-open-data][this one]].

In [ ]:
listings = pd.read_csv('data/listings_summary_dec18.csv')

- Interestingly, description would be a bad context. Rather, we need to look at how users describe a hotel!

In [ ]:
selected_columns = ["address","name","reviews/0/comments","reviews/1/comments","reviews/2/comments","reviews/3/comments","reviews/4/comments","reviews/5/comments","reviews/6/comments","reviews/7/comments","reviews/8/comments","reviews/9/comments"]

In [ ]:
dataset = dataset[selected_columns]

In [ ]:
dataset.head()

In [ ]:
dataset.shape

## Missing Value Handling

In [ ]:
dataset.describe()

In [ ]:
dataset.info()

In [ ]:
dataset.dropna()

In [ ]:
dataset.info()

## Create Document

In [ ]:
def row_to_tagged_document(row):
    comments = [str(row[f'reviews/{i}/comments']) for i in range(10)]
    words = ' '.join(comments).split()  
    tags = [row['name']]
    return TaggedDocument(words=words, tags=tags)

In [ ]:
tagged_documents = dataset.apply(row_to_tagged_document, axis=1).tolist()

In [ ]:
tagged_documents

## Train Doc2Vec

In [ ]:
%pip install gensim 

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

model = Doc2Vec(vector_size=50,  # Dimensionality of the document vectors
                window=2,         # Maximum distance between the current and predicted word within a sentence
                min_count=1,      # Ignores all words with total frequency lower than this
                workers=4,        # Number of CPU cores to use for training
                epochs=20)        # Number of training epochs

# Build the vocabulary
model.build_vocab(tagged_documents)

# Train the model
model.train(tagged_documents, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
model.save("doc2vec_model")

## Text Classification

In [ ]:
doc_vectors = [model.dv[idx] for idx in range(530)] 

In [ ]:
labels = dataset["name"][:530]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(doc_vectors, labels, test_size=0.2, random_state=42)

### Trying logistic regression

In [ ]:
from sklearn.preprocessing import StandardScaler
# Initialize and train a logistic regression classifier
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train_scaled, y_train)
# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

### Trying Decision Tree

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

classifier = DecisionTreeClassifier(random_state=42)
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


### KNN Classifier

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = knn_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Use the trained model to predict the class for a new vector
new_vector = X_train[0]
predicted_class = knn_classifier.predict([new_vector])
print(f"Predicted Class for the New Vector: {predicted_class}\nActual Class: {y_train.values[0]}")

### No Model, Just document searching?

In [ ]:
def find_relevant_documents(input_string, model, tagged_documents):
    # Infer the vector representation of the input string
    input_vector = model.infer_vector(input_string.split())

    # Find similar documents
    similar_documents = model.dv.most_similar([input_vector], topn=5)  # Adjust 'topn' as needed

    # Print the results
    print("Input String:", input_string)
    print("Relevant Documents:")
    for doc_tag, similarity in similar_documents:
        # Find the TaggedDocument based on its tag
        relevant_document = next(doc for doc in tagged_documents if doc.tags == doc_tag)
        
        print(f"Document Tag: {doc_tag}, Similarity: {similarity}")
        print("Document Content:", relevant_document.words)
        print()

tokens = "A big beautiful room".split()

new_vector = model.infer_vector(tokens)
sims = model.docvecs.most_similar([new_vector]) #gives you top 10 document tags and their cosine similarity

In [ ]:
sims

In [ ]:
for docId, probablity in sims:
    for words, tags in tagged_documents:
        if docId in tags:
            print(f"------------{docId}------------")
            print(" ".join(words))

- I have a feeling that KNN approach might work. Let us try that.

In [ ]:
from sklearn.neighbors import NearestNeighbors


def find_relevant_documents_knn(input_string, model, vectors, tagged_documents):
    # Infer the vector representation of the input string
    input_vector = model.infer_vector(input_string.split())

    # Use k-Nearest Neighbors to find similar vectors
    knn = NearestNeighbors(n_neighbors=5, metric='cosine')  # Adjust 'n_neighbors' as needed
    knn.fit(vectors)

    # Find indices and distances of k-neighbors
    _, indices = knn.kneighbors([input_vector])

    # Print the results
    print("Input String:", input_string)
    print("Relevant Documents:")
    for idx in indices.flatten():
        print(f"Document ID: {idx}")
        print("Document Content:", " ".join(tagged_documents[idx].tags))
        print("Document Content:", " ".join(tagged_documents[idx].words))
        print()

# Example input string
input_string = "A big beautiful room"

# Find relevant documents using KNN
find_relevant_documents_knn(input_string, model, doc_vectors, tagged_documents)


In [ ]:
!jupyter nbconvert --to script config_template.ipynb